In [4]:
import sqlite3
from google.cloud import bigquery

In [5]:
db_path = r'/mnt/disks/disk-1/d_light/sfc.db'


In [6]:
client = bigquery.Client()

In [12]:
table_names = [
    ["`afcproj.files_dupe.Payments_2020_11_17`", 'Payments'],
    ["`afcproj.files_dupe.Contracts_20201117`", 'Contracts'],
    ["`afcproj.files_dupe.Adjustments_2020_11_17`", 'Adjustments'],
]

In [27]:
## 1) Check the number of entries is the same
sqls = ["""
        SELECT count(*) as cnt
        FROM {}
        WHERE 
        paymentStatusTypeEntity != 'REFUSED'
        and
        PaymentResultTypeEntity != 'PAYMENT_FREE_CREDIT'
        """,
        """
        SELECT count(*) as cnt
        FROM {}
        WHERE PaymentMethod = 'FINANCED'
        """,
        """
        SELECT count(*) as cnt
        FROM {}
        """,
    ]

conn = sqlite3.connect(db_path)
cur = conn.cursor()

for (bq, sq), sql in zip(table_names, sqls):
    print(
    "number of rows in BiqQuery = {:,.0f} \nnumber of rows in sqlite = {:,.0f}".format(
        next(iter(client.query(sql.format(bq)).result()))[0], #104,857,117 for bigquery
        cur.execute(sql.format(sq)).fetchall()[0][0]
        )
    )

conn.close()
    

number of rows in BiqQuery = 104857117 
number of rows in sqlite = 106141468
number of rows in BiqQuery = 722033 
number of rows in sqlite = 728262
number of rows in BiqQuery = 124074 
number of rows in sqlite = 979251


In [28]:
## 2) Check the sum of amounts is the same
sqls = ["""
        SELECT sum(AmountPaid) as total
        FROM {}
        WHERE 
        paymentStatusTypeEntity != 'REFUSED'
        and
        PaymentResultTypeEntity != 'PAYMENT_FREE_CREDIT'
        """,
        """
        SELECT sum(Price) as total
        FROM {}
        WHERE PaymentMethod = 'FINANCED'
        """,
        """
        SELECT sum(Amount) as total
        FROM {}
        """,
    ]

conn = sqlite3.connect(db_path)
cur = conn.cursor()

for (bq, sq), sql in zip(table_names, sqls):
    print(
    "total amount in BiqQuery = {:,.0f} \ntotal amount in sqlite = {:,.0f}".format(
        next(iter(client.query(sql.format(bq)).result()))[0], #104,857,117 for bigquery
        cur.execute(sql.format(sq)).fetchall()[0][0]
        )
    )

conn.close()
    

total amount in BiqQuery = 12325815738 
total amount in sqlite = 12459691170
total amount in BiqQuery = 19875984753 
total amount in sqlite = 20062576814
total amount in BiqQuery = 3144721278 
total amount in sqlite = 25921336501
